# Introduction to Support Vector Machines with a Trading Example

This notebook is about how to implement a Support Vector Machines model in a financial markets example, taken from [this blog post on 'Towards Data Science'](https://towardsdatascience.com/intro-to-support-vector-machines-with-a-trading-example-1d4a7997ced6).

In [1]:
from bs4 import BeautifulSoup
import datetime, json
import numpy as np
import pandas as pd
import requests, time
import warnings
warnings.simplefilter('ignore')

In [2]:
# requires the TA-lib package
import talib as ta
from talib import MA_Type

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm

In [4]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker

In [5]:
Dow_Page = requests.get('https://finance.yahoo.com/quote/%5EDJI/components?p=%5EDJI')
Dow_Content = Dow_Page.content

soup = BeautifulSoup(Dow_Content)

data = list(soup.findAll("td",{"class":"Py(10px) Ta(start) Pend(10px)"}))

In [6]:
Ticker_List = []
for i in data:
    TempData = str(i)
    if "title" in TempData:
        TempData = TempData[TempData.find("title"):]
        TempData = TempData[TempData.find(">")+1:TempData.find("<")]
        Ticker_List.append(TempData)
    else:
        continue